# Importando datos de Twiiter en MongoDB

In [ ]:
import pymongo 
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError
import json

In [ ]:
# connect to database
connection = MongoClient('localhost', 27017)

In [ ]:
# Borramos la base de datos
connection.drop_database('twitter')

# Pymongo permite una sintaxis practicamente igual que la original
db = connection.twitter

In [ ]:
def insert_tweet(tweet_json):
   
    # Simplificamos la estructura user_mentions a un array de nombres de usuario
    user_mencioned_list = []
    for user_mencioned in tweet_json['user_mentions']:
        user_mencioned_list.append(user_mencioned["screen_name"])
        user_mencioned["_id"] = user_mencioned["screen_name"]
        try:
            db.users.insert_one(user_mencioned)
        except DuplicateKeyError:
            continue
    tweet_json['user_mentions'] = user_mencioned_list
                 
    # Creamos un documento por hashtag utilizando el operador $inc para incrementar el contador
    for entity in tweet_json['entities']:
        db.hashtags.find_and_modify(query = {"_id" : entity}, 
                                    update ={ "$inc": { "count": 1 } } , 
                                    upsert = True)
       
    
    #Utilizamos como _id de la colección el ID del Tweet
    db.tweets.replace_one({"_id" : tweet_json["id_str"]}, tweet_json, upsert=True)
    
    #Insertamos el documento dentro de la colección de usuarios, utilizando como _id el campo screen_name
    user_json = tweet_json['user']
    db.users.replace_one({"_id" : user_json["screen_name"]}, user_json, upsert=True) 

    
    #En el caso de que el Tweet tenga un Tweet padre (retweet) lo almenamos como documento independiente 
    if 'retweeted_status' in tweet_json:
        insert_tweet(tweet_json['retweeted_status'])

In [ ]:
tweets_data_path = '../data/tweets.json'

tweets_file = open(tweets_data_path, "r")
for line in tweets_file:
    tweet_json = json.loads(line)
    insert_tweet(tweet_json)


![png](./images/Mogodb%20Model.png)

## Contando el número de registros

In [ ]:
print "Número de Tweets: " , db.tweets.count()
print "Número de Usuarios: " , db.users.count()
print "Número de Hashtags: " , db.hashtags.count()

## Encontrar un Tweet que tenga Hashtags

In [ ]:
db.tweets.find_one({"entities.1" : {"$exists" : True }})

## Usuarios más populares

In [ ]:
cursor_users = db.users.find({"followers_count": {"$exists" : True }}).sort("followers_count", pymongo.DESCENDING).limit(10)

for user in cursor_users:
    print user["screen_name"], "-", user["followers_count"]

## Usuarios que más tweets tienen

In [ ]:
cursor = db.tweets.aggregate([
        {"$group" : { "_id" : "$user.screen_name", "count" : { "$sum" : 1}}},
        {"$sort" : {"count" : -1}},
        {"$limit" : 10}
    ])

for row in cursor:
    print row["_id"],"-", row["count"]

## Hashtags más populares

In [ ]:
for hashtag in db.hashtags.find({}).sort("count", pymongo.DESCENDING).limit(10):
    print hashtag["_id"], "-", hashtag["count"]


## Tweets de un determinado Hashtag

In [ ]:
for tweet in db.tweets.find({ "entities" : "NoSQL"}).limit(10):
    print tweet["user"]["screen_name"], "-", tweet["text"]

## Tweets de un determinado usuario

In [ ]:
for tweet in db.tweets.find({ "user.screen_name" : "MAGISTR_OM"}).limit(10):
    print tweet["user"]["screen_name"], "-", tweet["text"]

## Usuarios mas mencionados

In [ ]:
cursor = db.tweets.aggregate([
        {"$unwind" : "$user_mentions"},
        {"$group" : { "_id" : "$user_mentions", "count" : { "$sum" : 1}}},
        {"$sort" : {"count" : -1}},
        {"$limit" : 10}
    ])

for row in cursor:
    print row["_id"],"-", row["count"]

## Tweets donde se menciona a un usuario

In [ ]:
for tweet in db.tweets.find({ "user_mentions" : "couchbase"}).limit(10):
    print tweet["user"]["screen_name"], "-", tweet["text"]